In [ ]:
import gdown
import tensorflow as tf
import numpy as np
import os
import random
import networkx as nx
import csv

!mkdir datasets
!pip install pyvis
from pyvis.network import Network

#############################################################################################
# Youtube DATASET DOWNLOADS  source: http://datasets.syr.edu/pages/datasets.html
#############################################################################################
!gdown https://drive.google.com/uc?id=12aGrbOZqVMfOP46X8lj5qwqQui4kbMjZ -O datasets/youtube_edges.csv

#############################################################################################
# Facebook DATASET DOWNLOADS source: https://github.com/fatemehsrz/Shortest_Distance/tree/master/data
#############################################################################################
# Download facebook dataset edgelist in txt format (extracted from mtx)
#!gdown https://drive.google.com/uc?id=1v03XWRternGLDpRfKbRGoMiVX3dpOW3G -O datasets/facebook_edges.txt

#############################################################################################
# Douban DATASET DOWNLOADS source: http://datasets.syr.edu/pages/datasets.html
#############################################################################################
# Download blogcatalog dataset edgelist in cvs format
!gdown https://drive.google.com/uc?id=1ssjgKF5WpiXcIk7DfF6BXwPoWkqr5rOS -O datasets/douban_edges.csv
!gdown https://drive.google.com/uc?id=174k2qDmDhXrKFivGD00jBWAvJp9b1kiq -O HARP.zip
!unzip HARP.zip

Downloading...
From: https://drive.google.com/uc?id=12aGrbOZqVMfOP46X8lj5qwqQui4kbMjZ
To: /content/datasets/youtube_edges.csv
38.7MB [00:00, 75.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ssjgKF5WpiXcIk7DfF6BXwPoWkqr5rOS
To: /content/datasets/douban_edges.csv
8.29MB [00:00, 50.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=174k2qDmDhXrKFivGD00jBWAvJp9b1kiq
To: /content/HARP.zip
13.6MB [00:00, 51.5MB/s]
Archive:  HARP.zip
   creating: HARP/
  inflating: HARP/default.walks.12   
   creating: HARP/example_graphs/
  inflating: HARP/default.walks.13   
  inflating: HARP/Facebook.edges     
  inflating: HARP/.DS_Store          
  inflating: __MACOSX/HARP/._.DS_Store  
  inflating: HARP/default.walks.9    
  inflating: HARP/default.walks.0    
  inflating: HARP/default.walks.7    
  inflating: HARP/LICENSE            
  inflating: HARP/requirements.txt   
   creating: HARP/bin/
  inflating: HARP/default.walks.6    
  inflating: HARP/default.walks.1    
   creating:

In [ ]:
landmark_technique = "random"
dataset = "douban"

In [ ]:
def read_data(dataset):
  if dataset == "facebook":
      G=nx.read_edgelist("./datasets/facebook_edges.txt")
  elif dataset == "blogcatalog":
      G = nx.read_edgelist('./datasets/blogcatalog_edges.csv', delimiter=',', nodetype=str, encoding="utf-8")
  elif dataset == "douban":
      G = nx.read_edgelist('./datasets/douban_edges.csv', delimiter=',', nodetype=int, encoding="utf-8")
      G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering="sorted")
      nx.write_edgelist(G,'./datasets/douban_edges.txt')
      #mapping = {}
      #for v in G.nodes():
      #  mapping[v] = str(v)
      #G = nx.relabel.relabel_nodes(G, mapping)
  elif dataset == "youtube":
      G = nx.read_edgelist('./datasets/youtube_edges.csv', delimiter=',', nodetype=str, encoding="utf-8")
      G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering="sorted")

      nx.write_edgelist(G, './datasets/youtube_edges.txt')
  elif dataset == "flickr":
      G=nx.read_edgelist("./datasets/flickr_edges.txt")
  else:
      print("Invalid dataset name")

  nodes = list(G.nodes())
  edges = list(G.edges())
  num_nodes = len(nodes)
  num_edges = len(edges)
  print("Number of nodes", num_nodes)
  print("Number of edges", num_edges)
  return G, nodes, edges, num_nodes, num_edges

G, nodes, edges, num_nodes, num_edges = read_data(dataset)

Number of nodes 154907
Number of edges 327094


In [ ]:
print(min(nodes), max(nodes))

0 154906


In [ ]:
!pip install deepwalk
from networkx.algorithms.community.modularity_max import greedy_modularity_communities
from HARP.src.harp import run_coarsening, train_embedding

def select_landmarks(num_nodes, landmark_technique,  nodes, is_new_data_split):
  print()
  if is_new_data_split:
    print("#### LANDMARK SELECTION ####")

    # Select number of landmark nodes
    if (num_nodes>10000):
      k1=3
      k2=1
    else:
      k1=100
      k2=11

    # Select training landmarks
    if landmark_technique == "random":
      print("Selecting landmarks randomly...")
      k1_nodes = random.sample(nodes,k1)
    elif landmark_technique == "coarsening":
      print("Coarsening graph...")
      %cd HARP
      recursive_node_assosiations=run_coarsening("../datasets/"+dataset+"_edges.txt", None, "edgelist")[1]
      #embeddings = train_embedding("line", "Facebook_HARP_line.npx", "../datasets/facebook_edges.txt", "network")
      %cd ..
      for rec_level in recursive_node_assosiations[::-1]:
        rec_l = list(rec_level.keys())
        #print(len(rec_l))
        if len(rec_l) >= k1:
          print("Selecting landmarks as coarsened graph...")
          k1_nodes = random.sample(rec_l, k1)
          k1_nodes = [str(node) for node in k1_nodes]

          break
    elif landmark_technique == "community_detection":
      c = list(greedy_modularity_communities(G))
      ratios = []
      sum = 0
      k1_nodes = []
      print("Clustering into communities...")
      print("number of communities: ",c)
      for i in range(len(c)):
          percentage = int((len(c[i])/num_nodes)*100)
          if percentage==0:
              percentage = 1
          k1_nodes.extend(random.sample(c[i],percentage))
    # I think this is only needed for HARP
    print("Number of training landmark nodes:",len(k1_nodes))
    remaining_nodes_train = list(set(nodes)-set(k1_nodes))
    print("number of nodes except training (landmark) nodes", len(remaining_nodes_train))

    # Select testing landmarks
    k2_nodes = random.sample(remaining_nodes_train,k2)

    print("Number of testing landmark nodes:",len(k2_nodes))
    remaining_nodes_test = list(set(nodes)-set(k2_nodes))
    print("number of nodes except testing (landmark) nodes", len(remaining_nodes_test))
    print(k1_nodes)
    return k1_nodes, k2_nodes, remaining_nodes_train, remaining_nodes_test
  else:
    pass

k1_nodes, k2_nodes, remaining_nodes_train, remaining_nodes_test = select_landmarks(num_nodes, landmark_technique, nodes, True) 

/content/HARP/src

#### LANDMARK SELECTION ####
Selecting landmarks randomly...
Number of training landmark nodes: 3
number of nodes except training (landmark) nodes 154904
Number of testing landmark nodes: 1
number of nodes except testing (landmark) nodes 154906
[100642, 105822, 55852]


In [ ]:
def generate_and_save_train_data(G, k1_nodes, is_new_data_split, dataset, landmark_technique):
  train_set = []
  if is_new_data_split:
    for u in k1_nodes:
      for v in remaining_nodes_train:
          if nx.has_path(G, u, v):
            shortest_path = nx.shortest_path(G, u, v)
            length = 1
            for i in range(len(shortest_path)-1):
              train_set.append((shortest_path[0], shortest_path[i+1], length))
              length +=1

    print("Size of total training set before omission:",len(train_set))

    f_train = open('./datasets/train_'+dataset+'_'+landmark_technique+'.txt', 'w')
    for i in range(len(train_set)): 
      if (1< train_set[i][2] <= 6):
        f_train.write(str(train_set[i][0])+' '+str(train_set[i][1])+' '+str(train_set[i][2]) )
        f_train.write('\n')
              
    f_train.close()
    print("Train file written")

generate_and_save_train_data(G, k1_nodes, True, dataset, landmark_technique)

Size of total training set before omission: 20027051
Train file written


In [ ]:
def generate_and_save_test_data(G, k2_nodes, is_new_data_split, dataset, landmark_technique):
  test_set = []
  if is_new_data_split:
    for u in k2_nodes:
      for v in remaining_nodes_test:
        if nx.has_path(G, u, v):
          shortest_path = nx.shortest_path(G,u,v)
          length = 1
          for i in range(len(shortest_path) - 1):
            test_set.append((shortest_path[0], shortest_path[i+1], length))
            length += 1
    print("Size of total training set before omission:",len(test_set))

    f_test = open('./datasets/test_'+dataset+'_'+landmark_technique+'.txt', 'w')
    for i in range(len(test_set)):
      if (1< test_set[i][2] <= 6):
        f_test.write(str(test_set[i][0])+' '+ str(test_set[i][1]) +' '+ str(test_set[i][2]) )
        f_test.write('\n')

    f_test.close()
    print("Test file written")
generate_and_save_test_data(G, k2_nodes, True, dataset, landmark_technique)

Size of total training set before omission: 6273941
Test file written


In [ ]:
 def visualise_graph_with_landmarks(G, k1_nodes, landmark_technique):
  net = Network(notebook=True)
  net.from_nx(G)
  # Coloring landmark nodes
  for node_id in k1_nodes:
    net.node_map[str(node_id)]['shape'] = 'box'
    net.node_map[str(node_id)]['color'] = 'red'
  net.save_graph('graph_'+dataset+'_'+landmark_technique+'.html')

visualise_graph_with_landmarks(G, k1_nodes, landmark_technique)

KeyError: ignored